## 📄 Managed Tables

### 🔹 What is a Managed Table?

A **Managed Table** is a table where:

- Both **metadata** and **data files** are fully managed by the **Hive Metastore**.
- Dropping the table deletes **both the metadata and the underlying data**.

---

### 🔹 Creating a Managed Table:

In this example, we create a table named `managed_default`:

```sql
CREATE TABLE managed_default (id INT, name STRING);
```

- Notice there is no `LOCATION` keyword, meaning:
   - The data will be stored in the **default Hive warehouse directory**.
   - Hive Metastore handles the storage path automatically.

In [0]:
USE CATALOG test;

CREATE TABLE managed_default
  (width INT, length INT, height INT);

INSERT INTO managed_default
VALUES (3 INT, 2 INT, 1 INT);

#### Exploring Metadata

Use the following command to inspect table details:

```sql
DESCRIBE EXTENDED managed_default;
```

- This shows:
   - **Table name**: `managed_default`
   - **Table Location**: Stored in the default Hive path (Location may show blank, storage is abstracted)
   - **Table Type**: Managed

In [0]:
DESCRIBE EXTENDED managed_default


## 📄 External Tables

### 🔹 What is an External Table?

An **External Table** is a table where:

- Only the **metadata** is managed by the **Hive Metastore**.
- The **actual data files** are stored **outside** of Hive's default directory and control.
- Dropping the table removes **only the metadata**, but **does NOT delete** the underlying data files.

---

### 🔹 Creating an External Table:

To create an external table, you must explicitly define the storage path using the `LOCATION` keyword.

Example:

```sql
CREATE EXTERNAL TABLE external_table (id INT, name STRING)
LOCATION 'your location';
```

- The data filters for external_table will be stored in the '/mnt/demo' directory.
- Hive Metastore records only the metadata and the tabl'es storage location.

In [0]:
CREATE EXTERNAL LOCATION my_ext_loc
URL 's3://your-bucket/data/'
WITH STORAGE CREDENTIAL my_credential;

CREATE TABLE my_catalog.my_schema.external_default
  (width INT, length INT, height INT)
LOCATION '@my_ext_loc/external_default';
  
INSERT INTO external_default
VALUES (3 INT, 2 INT, 1 INT);


#### Verifying Table Creation:

- You can confirm successful creation in the Data Explorer.
- Running the following command inspects table metadata:

```sql
DESCRIBE EXTENDED external_table;
```

- This shows:
   - **Table Type**: External
   - **Table Location**: Shows the path `/mnt/demo/` defined during creation

In [0]:
DESCRIBE EXTENDED external_default;


## 🗑️ Dropping Tables

### 🔹 Dropping Managed vs External Tables:

---

### 1️⃣ Dropping a Managed Table:

```sql
DROP TABLE managed_default;
```

- Outcome:
  - Both **metadata** and the **underlying data files** are **permanently deleted**.
  - Managed tables are fully controlled by the Hive Metastore, so deletion removes everything.
---

### 2️⃣ Dropping an External Table:

```sql
DROP TABLE external_table;
```

- Outcome:
  - **Only the metadata is removed** from the Hive Metastore.
  - **Data files (e.g., stored at `/mnt/demo`) remain intact**.
  - You retain control over the physical data files.

In [0]:
DROP TABLE managed_default;

In [0]:
DROP TABLE external_default;

In [0]:
%fs ls 'dbfs:/mnt/demo/external_default'

## 📂 Creating Schemas

### 🔹 Schema Creation:

You can create new schemas (also referred to as databases) in Databricks using either syntax:

```sql
CREATE DATABASE sales_schema;
CREATE SCHEMA marketing_schema;
```

If the location is available the file will have .db for database/schemas

In [0]:
CREATE SCHEMA new_default;

In [0]:
DESCRIBE DATABASE EXTENDED new_default;

#### Adding Tables to the New Schema:
Once your schema is created you can add managed and external tables inside it

In [0]:
USE new_default;

CREATE TABLE managed_new_default
  (width INT, length INT, height INT);
  
INSERT INTO managed_new_default
VALUES (3 INT, 2 INT, 1 INT);

-----------------------------------

CREATE TABLE external_new_default
  (width INT, length INT, height INT)
LOCATION 'dbfs:/mnt/demo/external_new_default';
  
INSERT INTO external_new_default
VALUES (3 INT, 2 INT, 1 INT);

In [0]:
DESCRIBE EXTENDED managed_new_default;

In [0]:
DESCRIBE EXTENDED external_new_default;

#### Dropping Tables in Custom Schemas:
Dropping tables in custom schemas will have the same outcomes as described previously.

- Managed Table:
  - **Outcome**: Both metadata and data files are deleted.
- External Table:
  - **Outcome**: Metadata is delted, but underyling data files remain.

In [0]:
DROP TABLE managed_new_default;
DROP TABLE external_new_default;

## 📂 Creating Schemas in Custom Location

### 🔹 Creating Schemas in Custom Locations:

You can define a custom storage path when creating a new schema using the `LOCATION` keyword.

**Example:**

```sql
CREATE SCHEMA new_schema
LOCATION '/mnt/custom/path';
```
- The schema metadata is stored in the Hive Metastore.
- The actual data files for managed tables will be stored at the specified custom location.
---
### 🔹 Behavior Consistency:
- **Managed Tables**: Data files are stored inside the custom schema's defined location.
- **External Tables**: You can still specify a separate `LOCATION` for each table as needed.

The core behaviors remain the same:
- Dropping a **managed table** deletes both metadata and data files.
- Dropping an **external table** deletes only metadata, leaving the data files intact.

In [0]:
CREATE SCHEMA custom
LOCATION 'dbfs:/Shared/schemas/custom.db';

In [0]:
DESCRIBE DATABASE EXTENDED custom;

In [0]:
USE custom;

CREATE TABLE managed_custom
  (width INT, length INT, height INT);
  
INSERT INTO managed_custom
VALUES (3 INT, 2 INT, 1 INT);

-----------------------------------

CREATE TABLE external_custom
  (width INT, length INT, height INT)
LOCATION 'dbfs:/mnt/demo/external_custom';
  
INSERT INTO external_custom
VALUES (3 INT, 2 INT, 1 INT);

In [0]:
DESCRIBE EXTENDED managed_custom;

In [0]:
DESCRIBE EXTENDED external_custom;

In [0]:
DROP TABLE managed_custom;
DROP TABLE external_custom;